# PyPSA-USA Loads
This workbook comarpares the different load options agaisnt one another.

**Note, this must be run from the `scripts/` folder until PyPSA-USA is packaged**

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import math
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

import os

pd.options.plotting.backend = "plotly"

from build_demand import ReadCliu, ReadEfs, ReadEia, ReadEulp

## Helper Functions

In [ ]:
PLOTLY_COLORS = {
    "EIA": "#636EFA",
    "EFS": "#EF553B",
    "RESSTOCK": "#00CC96",
    "COMSTOCK": "#AB63FA",
    "CLIU": "#FFA15A",
    "Difference": "#FF6692",
    "SYSTEM": "#FECB52",
    "NREL": "#00CC96",
}

## Electricity Only 

Loads for electricity only studies 

In [ ]:
eia = ReadEia("./../data/GridEmissions/EIA_DMD_2018_2024.csv").read_demand()
efs = ReadEfs("./../data/nrel_efs/EFSLoadProfile_Reference_Moderate.csv").read_demand()

In [ ]:
def format_electricity(df, year=2020):
    df = df.loc[df.index.get_level_values("snapshot").year == year]
    df.index = df.index.droplevel(["sector", "subsector", "fuel"])
    df = pd.DataFrame(df.sum(axis=1), columns=["Load_MW"]).reset_index()
    return df.groupby(["snapshot"]).sum()


def plot_electricity(df, freq, cumsum):
    if cumsum:
        data = df.cumsum().resample(freq).sum()
        title = f"Cummulative Demand (Resampled at {freq})"
    else:
        data = df.resample(freq).sum()
        title = f"Demand (Resampled at {freq})"
    diff = pd.DataFrame((data.EIA - data.EFS) / data.EIA * 100, columns=["Difference"])

    fig = make_subplots(
        rows=2,
        cols=1,
        subplot_titles=(title, f"Difference (Resampled at {freq})"),
    )
    for source in ("EIA", "EFS"):
        fig.add_trace(
            go.Bar(
                x=data.index,
                y=data[source],
                name=source,
                marker={"color": PLOTLY_COLORS[source]},
            ),
            row=1,
            col=1,
        )
    fig.add_trace(
        go.Bar(
            x=diff.index,
            y=diff["Difference"],
            name="Difference",
            marker={"color": PLOTLY_COLORS["Difference"]},
        ),
        row=2,
        col=1,
    )
    fig["layout"]["yaxis"]["title"] = "Demand (MW)"
    fig["layout"]["yaxis2"]["title"] = "Difference (%)"

    fig.update_layout(autosize=True, height=750)

    fig.show()

In [ ]:
eia_2020 = format_electricity(eia).rename(columns={"Load_MW": "EIA"})
efs_2020 = format_electricity(efs).rename(columns={"Load_MW": "EFS"})
loads = eia_2020.join(efs_2020, how="outer").fillna(0)

In [ ]:
plot_electricity(loads, freq="W", cumsum=False)

In [ ]:
plot_electricity(loads, freq="W", cumsum=True)

## Sector Coupling Studies 

In [ ]:
efs = ReadEfs("./../data/nrel_efs/EFSLoadProfile_Reference_Moderate.csv").read_demand()
resstock_files = os.listdir("./../data/eulp/res/")
resstock = ReadEulp(
    [f"./../data/eulp/res/{x}" for x in resstock_files if x.endswith(".csv")],
    stock="res",
).read_demand()
comstock_files = os.listdir("./../data/eulp/com/")
comstock = ReadEulp(
    [f"./../data/eulp/com/{x}" for x in comstock_files if x.endswith(".csv")],
    stock="com",
).read_demand()
cliu = ReadCliu(
    "./../data/industry_load/2014_update_20170910-0116.csv",
    profiles_filepath="./../data/industry_load/epri_industrial_loads.csv",
).read_demand()

### Helpers

In [ ]:
def adjust_snapshots(df, year=2018):
    assert df.index.name == "snapshot"
    df = df.reset_index()
    df["snapshot"] = df.snapshot.map(lambda x: x.replace(year=year))
    return df.groupby("snapshot").sum().sort_index()


def group_sector_snapshots(df):
    assert df.index.name == "snapshot"
    df = df.reset_index()
    return df.groupby("snapshot").sum().sort_index()


def filter_sector(df, sector):
    df = df.loc[
        (df.index.get_level_values("sector") == sector)
        & (df.index.get_level_values("fuel") == "electricity")
    ].copy()
    return df.droplevel(["sector", "subsector", "fuel"])


def filter_sector_snapshots(df, year=2018):
    return df.loc[df.index.get_level_values("snapshot").year == year]


def format_sector_data(df):
    return (
        df.reset_index()
        .melt(id_vars="snapshot", var_name="State")
        .set_index("snapshot")
    )


def plot_sector(efs, other, other_name, sector, freq, cumsum):
    if cumsum:
        efs_data = (
            efs[["value"]]
            .rename(columns={"value": "EFS"})
            .cumsum()
            .resample(freq)
            .sum()
        )
        other_data = (
            other[["value"]]
            .rename(columns={"value": other_name})
            .cumsum()
            .resample(freq)
            .sum()
        )
        sns_data = efs_data.join(other_data)
        title = f"Cummulative System Level {sector} Demand (Resampled at {freq})"
    else:
        efs_data = efs[["value"]].rename(columns={"value": "EFS"}).resample(freq).sum()
        other_data = (
            other[["value"]].rename(columns={"value": other_name}).resample(freq).sum()
        )
        sns_data = efs_data.join(other_data)
        title = f"System Level {sector} Demand (Resampled at {freq})"
    diff = pd.DataFrame(
        (efs_data.EFS - sns_data[other_name]) / sns_data.EFS * 100,
        columns=["Difference"],
    )

    fig = make_subplots(
        rows=2,
        cols=1,
        subplot_titles=(
            title,
            f"{sector} Difference (Resampled at {freq})",
        ),
    )
    for source in ("EFS", other_name):
        fig.add_trace(
            go.Bar(
                x=sns_data.index,
                y=sns_data[source],
                name=source,
                marker={"color": PLOTLY_COLORS[source]},
            ),
            row=1,
            col=1,
        )
    fig.add_trace(
        go.Bar(
            x=diff.index,
            y=diff["Difference"],
            name="Difference",
            marker={"color": PLOTLY_COLORS["Difference"]},
        ),
        row=2,
        col=1,
    )
    fig["layout"]["yaxis"]["title"] = "Demand (MW)"
    fig["layout"]["yaxis2"]["title"] = "Difference (%)"

    fig.update_layout(autosize=True, height=750)

    fig.show()


def plot_sector_state(efs, other, other_name, sector):

    state_efs_data = (
        efs.reset_index()
        .drop(columns="snapshot")
        .groupby("State")
        .sum()
        .rename(columns={"value": "EFS"})
    )
    state_other_data = (
        other.reset_index()
        .drop(columns="snapshot")
        .groupby("State")
        .sum()
        .rename(columns={"value": other_name})
    )
    state_data = state_efs_data.join(state_other_data).fillna(0)
    diff = pd.DataFrame(
        (state_data.EFS - state_data[other_name]) / state_data.EFS * 100,
        columns=["Difference"],
    )

    fig = make_subplots(
        rows=2,
        cols=1,
        subplot_titles=(
            f"{sector} State Level Electrical Demand",
            f"{sector} Difference",
        ),
    )

    for source in ("EFS", other_name):
        fig.add_trace(
            go.Bar(
                x=state_data.index,
                y=state_data[source],
                name=source,
                marker={"color": PLOTLY_COLORS[source]},
            ),
            row=1,
            col=1,
        )
    fig.add_trace(
        go.Bar(
            x=diff.index,
            y=diff["Difference"],
            name="Difference",
            marker={"color": PLOTLY_COLORS["Difference"]},
        ),
        row=2,
        col=1,
    )
    fig["layout"]["yaxis"]["title"] = "Demand (MW)"
    fig["layout"]["yaxis2"]["title"] = "Difference (%)"

    fig.update_layout(autosize=True, height=750)

    fig.show()

### Adjust to 2018 data

In [ ]:
efs_2018 = filter_sector_snapshots(efs)

### Residential Demand

In [ ]:
resstock_2018 = filter_sector(resstock, "residential")
resstock_2018 = adjust_snapshots(resstock_2018)
resstock_2018 = format_sector_data(resstock_2018)
efs_res_2018 = filter_sector(efs_2018, "residential")
efs_res_2018 = group_sector_snapshots(efs_res_2018)
efs_res_2018 = format_sector_data(efs_res_2018)

In [ ]:
plot_sector(
    efs_res_2018, resstock_2018, "RESSTOCK", "Residential", freq="W", cumsum=False
)

In [ ]:
plot_sector_state(efs_res_2018, resstock_2018, "RESSTOCK", "Residential")

### Commercial Demand 

In [ ]:
comstock_2018 = filter_sector(comstock, "commercial")
comstock_2018 = adjust_snapshots(comstock_2018)
comstock_2018 = format_sector_data(comstock_2018)
efs_com_2018 = filter_sector(efs_2018, "commercial")
efs_com_2018 = group_sector_snapshots(efs_com_2018)
efs_com_2018 = format_sector_data(efs_com_2018)

In [ ]:
plot_sector(
    efs_com_2018, comstock_2018, "COMSTOCK", "Commercial", freq="W", cumsum=False
)

In [ ]:
plot_sector_state(efs_com_2018, comstock_2018, "COMSTOCK", "Commercial")

### Res + Com Demand

In [ ]:
nrel_building_2018 = pd.concat(
    [
        resstock_2018.reset_index(),
        comstock_2018.reset_index(),
    ]
)
nrel_building_2018 = (
    nrel_building_2018.groupby(["snapshot", "State"])
    .sum()
    .reset_index()
    .set_index("snapshot")
)

efs_building_2018 = pd.concat(
    [
        efs_res_2018.reset_index(),
        efs_com_2018.reset_index(),
    ]
)
efs_building_2018 = (
    efs_building_2018.groupby(["snapshot", "State"])
    .sum()
    .reset_index()
    .set_index("snapshot")
)

In [ ]:
plot_sector(
    efs_building_2018, nrel_building_2018, "NREL", "Buildings", freq="W", cumsum=False
)

In [ ]:
plot_sector(
    efs_building_2018, nrel_building_2018, "NREL", "Buildings", freq="W", cumsum=True
)

In [ ]:
plot_sector_state(efs_building_2018, nrel_building_2018, "NREL", "Buildings")

### Industrial Demand 

In [ ]:
cliu_2018 = filter_sector(cliu, "industry")
cliu_2018 = adjust_snapshots(cliu_2018)
cliu_2018 = format_sector_data(cliu_2018)
efs_ind_2018 = filter_sector(efs_2018, "industry")
efs_ind_2018 = group_sector_snapshots(efs_ind_2018)
efs_ind_2018 = format_sector_data(efs_ind_2018)

In [ ]:
plot_sector(efs_ind_2018, cliu_2018, "CLIU", "Industry", freq="W", cumsum=False)

In [ ]:
plot_sector_state(efs_ind_2018, cliu_2018, "CLIU", "Industry")

## System Level Sector Coupling

In [ ]:
efs_trn_2018 = filter_sector(efs_2018, "transport")
efs_trn_2018 = group_sector_snapshots(efs_trn_2018)
efs_trn_2018 = format_sector_data(efs_trn_2018)

In [ ]:
system_sectors = pd.concat(
    [
        resstock_2018.reset_index(),
        comstock_2018.reset_index(),
        cliu_2018.reset_index(),
        efs_trn_2018.reset_index(),
    ]
)
system_sectors = (
    system_sectors.groupby(["snapshot", "State"])
    .sum()
    .reset_index()
    .set_index("snapshot")
)

In [ ]:
efs_2018 = efs.loc[
    (efs.index.get_level_values("snapshot").year == 2018)
    & (efs.index.get_level_values("fuel") == "electricity")
]
efs_2018.index = efs_2018.index.droplevel(["sector", "subsector", "fuel"])
efs_2018 = format_sector_data(efs_2018)

In [ ]:
plot_sector(efs_2018, system_sectors, "SYSTEM", "Total", freq="W", cumsum=False)

In [ ]:
plot_sector_state(efs_2018, system_sectors, "SYSTEM", "Total")